In [1]:
import nltk
import re
import string
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from collections import defaultdict
import random

In [34]:
def preprocessing(line, negative_contruct:list, smile_positive:list, smile_negative:list, ps:'Porter Stemmer', stop_words:list):
    line = re.sub(r'http\S+', '', line)  # remove URL
    line = re.sub('@[\w_]+', '', line)   # remove USER_MENTIONS
    line = re.sub('#[\w]+', '', line)   # remove hashtags
    line = line.split()
    for i, word in enumerate(line):     # replace negative constructs with not
#         print(line)
        if line[i] in stop_words:
            line[i] = ''
        if word in negative_construct:
            line[i]="not"
        elif word in smile_positive:    #replacing emojis with class name
            line[i] = "smile_positive"
        elif word in smile_negative:
            line[i] = "smile_negative"
        else:
            line[i] = ps.stem(word)      # stemming
        
        
#             print('yes')
        line[i] = line[i].lower()
#         print(line)
    line = ' '.join(line)
#     print(line)

    translator = str.maketrans('', '', string.punctuation) # remove punctuations
    line = line.translate(translator)
    line = ' '.join(line.split())
    return line

In [9]:
file ='./Dataset.txt'
negative_construct = [ "can't", "wouldn't", "wasn't", "hadn't", "never", "won't"]

smile_pos = """:‑) :-] :-3 :-> 8-) :-} :o) :c) :^) =] =) :) :] :3 :> 8) :} :‑D :D 8‑D 8D x‑D xD
X‑D XD =D =3 B^D :-)) :'‑) :') :‑O :O :‑o :o :-0 8‑0 >:O :-* :* :× ;‑) ;) *-) *)
;‑] ;] ;^) :‑, ;D :‑P :P X‑P XP x‑p xp :‑p :p :‑Þ :‑Þ :‑þ :þ :Þ :Þ :‑b :b d: =p
>:P O:‑) O:) 0:‑3 0:3 0:‑) 0:) 0;^) |;‑) :‑J #‑) %‑) %) <3 @};- @}->--
@}‑;‑'‑‑‑ @>‑‑>‑‑""".split()

smile_neg = """:‑( :( :‑c :c :‑< :< :‑[ :[ :-|| >:[ :{ :@ >:( :'‑( :'( D‑': D:< D: D8 D; D= DX :‑/
:/ :‑. >:\ :L =L :S :‑| :| :‑X :X :‑# :# :‑& :& >:‑) >:) }:‑) }:) 3:‑) 3:) >;) ',:-l
',:-| >_> <_< <\3 </3
""".split()

ps = PorterStemmer()

stop_words = set(stopwords.words('english'))

In [4]:
lines = []
with open(file, encoding='utf-8') as f:
    for line in f:
        line = preprocessing(line, negative_construct, smile_pos, smile_neg, ps, stop_words)
        lines.append(line)

In [39]:
s = ":) won't running I am the best won't i've :("
print(preprocessing(s, negative_construct, smile_pos, smile_neg, ps, stop_words))

smilepositive not run i am the best not iv smilenegative


In [38]:
print(lines[:10])

['1 da vinci code book awesome', '1 thi wa first clive cussler iv ever read even book like relic da vinci code plausibl this', '1 like da vinci code lot', '1 like da vinci code lot', '1 i like da vinci code ultimatli seem hold it own', '1 that even exagger midnight went walmart buy da vinci code amaz course', '1 i love da vinci code i want someth better different', '1 thought da vinci code wa great kite runner', '1 da vinci code actual good movie', '1 i thought da vinci code wa pretti good book']


In [43]:
def get_train_test(lines: 'list of sentences', percent):
    data = []
    for line in lines:
        outcome = int(line.split()[0])
        sentence = ' '.join(line.split()[1:])
        temp = [sentence, outcome]
        data.append(temp)
    
    random.shuffle(data)
    split_index = int(percent * len(data))
    train_data = data[:split_index]
    test_data = data[split_index:]
    
    return train_data, test_data       

In [44]:
train_data, test_data = get_train_test(lines, 0.75)

In [45]:
print(len(train_data), len(test_data))


5314 1772


In [46]:
def generate_probabilities(dataset:'list[0] = sentence, list[1]= outcome'):
    prob = defaultdict(int)
    outcome_set = set()
    # first store the counts of (word, outcome)
    for sentence, outcome in dataset:
        outcome_set.add(outcome)
        prob[(outcome, None)] += 1 # (outcome, None) gives count of the outcome
        for word in sentence.split():
            tup = (word, outcome)
            prob[tup] += 1
            
    # convert the counts to probabilities
    for tup, count in prob.items():
        if tup[1] is not None:
            prob[tup] = prob[tup] / prob[(tup[1], None)] # tup[1] is the outcome and (outcome, None) has count of the outcome
    # convert count of outcome to prob
    for outcome in outcome_set:
        prob[(outcome, None)] = prob[(outcome, None)] / len(dataset)
    return (prob, outcome_set)

In [47]:
probabilities, possible_outcomes = generate_probabilities(train_data)

In [49]:
print(probabilities[(1, None)])
print(probabilities[(0, None)])
print(possible_outcomes)
print(probabilities)

0.5607828377869778
0.4392171622130222
{0, 1}
defaultdict(<class 'int'>, {(1, None): 0.5607828377869778, ('da', 1): 0.2436241610738255, ('vinci', 1): 0.24395973154362416, ('code', 1): 0.24328859060402686, ('wa', 1): 0.2087248322147651, ('awesom', 1): 0.13020134228187918, ('movie', 1): 0.052684563758389265, (0, None): 0.4392171622130222, ('da', 0): 0.32519280205655526, ('vinci', 0): 0.3260497000856898, ('code', 0): 0.32519280205655526, ('suck', 0): 0.2776349614395887, ('be', 0): 0.027849185946872322, ('brokeback', 1): 0.25033557046979865, ('mountain', 1): 0.2493288590604027, ('beautiful', 1): 0.024161073825503355, ('i', 1): 0.6422818791946309, ('love', 1): 0.4436241610738255, ('madly', 1): 0.0003355704697986577, ('i', 0): 0.408311910882605, ('hate', 0): 0.21936589545844046, ('harri', 0): 0.32305055698371893, ('potter', 0): 0.32476435304198803, ('daniel', 0): 0.027420736932305057, ('wotshisfac', 0): 0.027420736932305057, ('need', 0): 0.028277634961439587, ('fuck', 0): 0.05869751499571551,

In [61]:
def bayes(probabilities:'dictionary of tuple to prob', sentence, possible_outcomes):
    best_outcome = -1
    best_prob = -1
    for outcome in possible_outcomes:
        cur_prob = 1
        for word in sentence.split():
            tup = (word, outcome)
            cur_prob *= probabilities[tup]
        cur_prob *= probabilities[(outcome, None)]
        if cur_prob > best_prob:
            best_prob = cur_prob
            best_outcome = outcome
    return best_outcome
    

In [62]:
correct_and_predicted = [] # list which stores correct outcome and predicted outcome for accuracy, f1 etc. temp[0] is correct. temp[1] is predicted
for sentence, outcome in test_data:
    predict = bayes(probabilities, sentence, possible_outcomes)
    temp = [outcome, predict]
    correct_and_predicted.append(temp)

In [66]:
true_positive = 0
true_negative = 0
false_positive = 0
false_negative = 0
for given, predicted in correct_and_predicted:
    if given == 1 and predicted == 1:
        true_positive += 1
    elif given == 1 and predicted == 0:
        false_negative += 1
    elif given == 0 and predicted == 0:
        true_negative += 1
    elif given == 0 and predicted == 1:
        false_positive += 1

    
precision = true_positive / (true_positive + false_positive) # true_positive out of all the predicted positive
recall = true_positive / (true_positive + false_negative) # ture_positive out of all the actual positive
f1_score = (2 * precision * recall) / (precision + recall)
accuracy = (true_positive + true_negative) / len(correct_and_predicted) # total correct predicted
print(precision, recall, f1_score, accuracy)

0.9978331527627302 0.9073891625615763 0.9504643962848298 0.945823927765237
